# Storing Particle Shape

## Overview

### Questions

* How can I store particle shape for use with visualization tools?

### Objectives

* Demonstrate logging **type_shapes** to a **GSD** file.
* Explain that OVITO can read this information.

## Boilerplate code

In [1]:
import hoomd
import gsd.hoomd

Remove files that this notebook will generate.

In [2]:
import os
os.path.join(os.getcwd(), 'trajectory.gsd')
![ -e "$fn" ] && "$fn"

## Particle Shape

HPMC integrators and some anisotropic MD pair potentials model particles that have a well defined shape.
You can save this shape definition to a **GSD** file for use in analysis and visualization workflows.
In particular, [OVITO](https://www.ovito.org/) will read this shape information and render particles appropriately.

## Define the Simulation

This section executes the hard particle simulation from a previous tutorial. 
See [*Introducing HOOMD-blue*](../00-Introducing-HOOMD-blue/00-index.ipynb) for a complete description of this code.

In [3]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
mc = hoomd.hpmc.integrate.ConvexPolyhedron(seed=2)
mc.shape['octahedron'] = dict(vertices=[(-0.5, 0, 0),
                                         (0.5, 0, 0),
                                         (0, -0.5, 0),
                                         (0, 0.5, 0),
                                         (0, 0, -0.5),
                                         (0, 0, 0.5)])
sim.operations.integrator = mc
sim.create_state_from_gsd(filename='../00-Introducing-HOOMD-blue/compressed.gsd')
sim.run(0)

## Logging particle shape to a GSD file

The **type_shapes** loggable quantity is a representation of the particle shape for each type following the [**type_shapes** specification](https://gsd.readthedocs.io/en/stable/shapes.html) for the **GSD** file format.
In HPMC simulations, the integrator provides **type_shapes**:

In [4]:
mc.loggables

{'state': 'state',
 'map_overlaps': 'sequence',
 'overlaps': 'scalar',
 'translate_moves': 'sequence',
 'rotate_moves': 'sequence',
 'mps': 'scalar',
 'type_shapes': 'object'}

In [5]:
mc.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0,
  'vertices': [[-0.5, 0, 0],
   [0.5, 0, 0],
   [0, -0.5, 0],
   [0, 0.5, 0],
   [0, 0, -0.5],
   [0, 0, 0.5]]}]

Add the **type_shapes** quantity to a **Logger**.

In [6]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

Write the simulation trajectory to a **GSD** file along with the logged quantities:

In [7]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(10000),
                             mode='xb',
                             filter=hoomd.filter.All(),
                             log=logger)
sim.operations.writers.append(gsd_writer)

Run the simulation:

In [8]:
sim.run(20000)

As discussed in a previous section, delete the simulation so it is safe to open the GSD file for reading in the same process.

In [9]:
del sim, gsd_writer, logger, mc, cpu

## Reading logged shapes from a GSD file

You can access the shape from scripts using the `gsd` package:

In [10]:
traj = gsd.hoomd.open('trajectory.gsd', 'rb')

**type_shapes** is a special quantity available via `particles.type_shapes` rather than the `log` dictionary:

In [11]:
traj[0].particles.type_shapes

[{'type': 'ConvexPolyhedron',
  'rounding_radius': 0,
  'vertices': [[-0.5, 0, 0],
   [0.5, 0, 0],
   [0, -0.5, 0],
   [0, 0.5, 0],
   [0, 0, -0.5],
   [0, 0, 0.5]]}]

Open the file in OVITO and it will read the shape definition and render particles appropriately.

In this section, you have logged particle shape to a GSD file during a simulation so that visualization and analysis tools can access it.
The next section shows how to write formatted output.

[Previous section](02-Saving-Array-Quantities.ipynb) / [Next Section](04-Writing-Formatted-Output.ipynb)